## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Hithadhoo,MV,-0.6000,73.0833,27.96,78,100,5.35,overcast clouds
1,Punta Arenas,CL,-53.1500,-70.9167,9.00,61,75,5.14,broken clouds
2,Dharchula,IN,29.8500,80.5333,11.98,47,97,1.34,overcast clouds
3,Albany,US,42.6001,-73.9662,-3.33,64,27,1.19,scattered clouds
4,Kloulklubed,PW,7.0419,134.2556,30.00,70,75,4.63,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is your perferred minimum temperature for your desired vacation place? "))
max_temp = float(input("What is your perferred maximum temperature for your desired vacation place? "))

In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
perferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"]>= min_temp) & (city_data_df["Max Temp"]<= max_temp)]
perferred_cities_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Hithadhoo,MV,-0.6000,73.0833,27.96,78,100,5.35,overcast clouds
4,Kloulklubed,PW,7.0419,134.2556,30.00,70,75,4.63,broken clouds
6,Georgetown,MY,5.4112,100.3354,31.00,54,20,1.54,few clouds
8,Atuona,PF,-9.8000,-139.0333,25.98,74,3,4.84,clear sky
12,Bathsheba,BB,13.2167,-59.5167,26.00,65,40,8.23,scattered clouds


In [10]:
# 4a. Determine if there are any empty rows.
perferred_cities_df.isnull().sum()
perferred_cities_df.count()

City                   149
Country                149
Lat                    149
Lng                    149
Max Temp               149
Humidity               149
Cloudiness             149
Wind Speed             149
Current Description    149
dtype: int64

In [14]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_perferred_cities_df = perferred_cities_df.dropna()
clean_perferred_cities_df.count()

City                   149
Country                149
Lat                    149
Lng                    149
Max Temp               149
Humidity               149
Cloudiness             149
Wind Speed             149
Current Description    149
dtype: int64

In [15]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_perferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = np.nan
hotel_df.head(10)


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hithadhoo,MV,27.96,overcast clouds,-0.6000,73.0833,NaN
4,Kloulklubed,PW,30.00,broken clouds,7.0419,134.2556,NaN
6,Georgetown,MY,31.00,few clouds,5.4112,100.3354,NaN
8,Atuona,PF,25.98,clear sky,-9.8000,-139.0333,NaN
12,Bathsheba,BB,26.00,scattered clouds,13.2167,-59.5167,NaN
15,Souillac,MU,25.00,scattered clouds,-20.5167,57.5167,NaN
16,Kathu,TH,29.00,few clouds,7.9178,98.3332,NaN
17,Neiafu,TO,27.00,light rain,-18.6500,-173.9833,NaN
23,Avarua,CK,29.00,few clouds,-21.2078,-159.7750,NaN
24,Butaritari,KI,27.13,light rain,3.0707,172.7902,NaN


In [16]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params =params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,"Hotel Name"] = hotels["results"][0]["name"]
    except:
        print("Hotel not found... skipping...")
        

Hotel not found... skipping...
Hotel not found... skipping...
Hotel not found... skipping...
Hotel not found... skipping...
Hotel not found... skipping...
Hotel not found... skipping...


In [22]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()



City                   149
Country                149
Max Temp               149
Current Description    149
Lat                    149
Lng                    149
Hotel Name             143
dtype: int64

In [23]:
hotel_df_clean = hotel_df.dropna()
hotel_df_clean.count()

City                   143
Country                143
Max Temp               143
Current Description    143
Lat                    143
Lng                    143
Hotel Name             143
dtype: int64

In [24]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df_clean.to_csv(output_data_file, index_label="City_ID")

In [29]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °C</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df_clean.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df_clean[["Lat", "Lng"]]

In [30]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))